In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!apt install chromium-chromedriver
!pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 75.5 MB of archives.
After this operation, 256 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 83.0.4103.61-0ubuntu0.18.04.1 [1,119 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 83.0.410

In [13]:
from bs4 import BeautifulSoup
import pandas as pd

genre_dic = {}

def makeUrl(dateList,i, kinds):
  if kinds == 'playstore': 
    url = 'https://www.mobileindex.com/app/get_rank_all?rt=r&mk=2&c=kr&t=app&rs=100&d=' + dateList[i]
  else:
    url = 'https://www.mobileindex.com/app/get_rank_all?rt=r&mk=1&c=kr&t=app&rs=100&d=' + dateList[i]
  return url

def getGenreUrl(url):
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url)
  #driver.implicitly_wait(3)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  genreURL = soup.select('div.item-info > a')
  return genreURL

def get_genre(url,appname):
  if appname in genre_dic:
    return genre_dic[appname]
  dr = webdriver.Chrome('chromedriver',options=options)
  dr.get(url)
  #dr.implicitly_wait(3)
  ht = dr.page_source
  so = BeautifulSoup(ht, 'html.parser')
  genre = so.select('tr.text-center')[0].text
  index = genre.find('2')
  genre_dic[appname] = genre[13:(index-1)]
  return genre[13:(index-1)]

def separateAppRank(rank):
  free_rank = []
  pay_rank = []
  sales_rank = [] 
  for j in range(len(rank)):
    if len(rank[j]) == 100:
      free_rank.append(rank[j][0::2])
      pay_rank.append(rank[j][1::2])
    elif len(rank[j]) == 150:
      free_rank.append(rank[j][0::3])
      pay_rank.append(rank[j][1::3])
      sales_rank.append(rank[j][2::3])
   #무료, 유료 순위만 존재
  if len(sales_rank) == 0:
    return free_rank, pay_rank
  #무료, 유료, 매출순위 존재
  else:
    return free_rank, pay_rank, sales_rank
  return

def getAppRank(url, year):
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url)
  #driver.implicitly_wait(3)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  notices = soup.select('span.appname')
  
  tmp = []
  for n in notices:
    tmp.append(n.text.strip())
  return tmp

In [14]:
#2019
appRank = []
date_2019 = ['2019-01-31','2019-02-28','2019-03-31','2019-04-30','2019-05-31','2019-06-30','2019-07-31','2019-08-31','2019-09-30','2019-10-31','2019-11-30','2019-12-31']
date_size = len(date_2019)
kinds = ['playstore','appstore']

for kind in kinds:
  appRank = []
  appGenre = []
  for i in range(date_size):
    url = makeUrl(date_2019,i,kind)
    tmp = getAppRank(url,2019)
    appRank.append(tmp)
    genreURL = getGenreUrl(url)
    genreList = list()
    for j in range(len(genreURL)):
      genreList.append(get_genre('https://www.mobileindex.com/'+genreURL[j]['href'],tmp[j]))
    appGenre.append(genreList)
    print(kind,' data of ',date_2019[i],' is done')

  
  if kind == 'playstore': 
    p_appRank_free_2019,p_appRank_pay_2019 = separateAppRank(appRank)
    print('sep is done')
    p_appGenre_free_2019,p_appGenre_pay_2019  = separateAppRank(appGenre)
    print('sep2 is done') 
  else:
    a_appRank_free_2019,a_appRank_pay_2019 = separateAppRank(appRank)
    print('sep is done')
    a_appGenre_free_2019,a_appGenre_pay_2019  = separateAppRank(appGenre)
    print('sep2 is done') 

KeyboardInterrupt: ignored

In [5]:
p_appRank_free_2019 = []
p_appRank_pay_2019 = []
a_appRank_free_2019 = []
a_appRank_pay_2019 = []

In [6]:
p_appRank_free_2019,p_appRank_pay_2019 = separateAppRank(appRank)
print('sep is done')
p_appGenre_free_2019,p_appGenre_pay_2019  = separateAppRank(appGenre)
print('sep2 is done')

TypeError: ignored

In [10]:
len(appRank)

12

In [ ]:
#2020
appRank = []
date_2020 = ['2020-01-31','2020-02-29','2020-03-31','2020-04-30','2020-05-31','2020-06-30']
date_size = len(date_2020)
kinds = ['playstore','appstore']

for kind in kinds:
  appRank = []
  appGenre = []
  for i in range(date_size):
    url = makeUrl(date_2020,i,kind)
    tmp = getAppRank(url,2020)
    appRank.append(tmp)
    genreURL = getGenreUrl(url)
    genreList = list()
    for j in range(len(genreURL)):
      genreList.append(get_genre('https://www.mobileindex.com/'+genreURL[j]['href'],tmp[j]))
    appGenre.append(genreList)
    print(kind,' date of ',date_2020[i],' is done')
  
  if kind == 'playstore': 
    p_appRank_free_2020,p_appRank_pay_2020,p_sales_rank = separateAppRank(appRank)
    print('sep is done')
    p_appGenre_free_2020,p_appGenre_pay_2020,p_sales_genre  = separateAppRank(appGenre)
    print('sep2 is done') 
  else:
    a_appRank_free_2020,a_appRank_pay_2020,a_sales_rank = separateAppRank(appRank)
    print('sep is done')
    a_appGenre_free_2020,a_appGenre_pay_2020,a_sales_genre  = separateAppRank(appGenre)
    print('sep2 is done') 

playstore  date of  2020-01-31  is done
playstore  date of  2020-02-29  is done
playstore  date of  2020-03-31  is done
playstore  date of  2020-04-30  is done
playstore  date of  2020-05-31  is done
playstore  date of  2020-06-30  is done


NameError: ignored

In [ ]:
p_appRank_free = list()
for i in range(len(p_appRank_free_2019)):
  p_appRank_free.append(p_appRank_free_2019[i])
  p_appRank_free.append(p_appGenre_free_2019[i])
for i in range(len(p_appRank_free_2020)):
  p_appRank_free.append(p_appRank_free_2020[i])
  p_appRank_free.append(p_appGenre_free_2020[i])

a_appRank_free = list()
for i in range(len(a_appRank_free_2019)):
  a_appRank_free.append(a_appRank_free_2019[i])
  a_appRank_free.append(a_appGenre_free_2019[i])
for i in range(len(a_appRank_free_2020)):
  a_appRank_free.append(a_appRank_free_2020[i])
  a_appRank_free.append(a_appGenre_free_2020[i])


In [ ]:
col = []
for i in range(12):
  col.append('19_'+str(i+1))
  col.append('19_'+str(i+1)+'_gen')
for i in range(6):
  col.append('20_'+str(i+1))
  col.append('20_'+str(i+1)+'_gen')

appRank_free_playstore_df = pd.DataFrame(p_appRank_free)
appRank_free_playstore_df = appRank_free_playstore_df.T
appRank_free_playstore_df.columns = col
appRank_free_playstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_free_playstore.csv', encoding='utf-8-sig')

appRank_free_appstore_df = pd.DataFrame(a_appRank_free)
appRank_free_appstore_df = appRank_free_appstore_df.T
appRank_free_appstore_df.columns = col
appRank_free_appstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_free_appstore.csv', encoding='utf-8-sig')

In [ ]:
p_appRank_pay = list()
for i in range(len(p_appRank_free_2019)):
  p_appRank_pay.append(p_appRank_free_2019[i])
  p_appRank_pay.append(p_appGenre_free_2019[i])
for i in range(len(p_appRank_free_2020)):
  p_appRank_pay.append(p_appRank_free_2020[i])
  p_appRank_pay.append(p_appGenre_free_2020[i])

a_appRank_pay = list()
for i in range(len(a_appRank_pay_2019)):
  a_appRank_pay.append(a_appRank_pay_2019[i])
  a_appRank_pay.append(a_appGenre_pay_2019[i])
for i in range(len(a_appRank_pay_2020)):
  a_appRank_pay.append(a_appRank_pay_2020[i])
  a_appRank_pay.append(a_appGenre_pay_2020[i])


In [ ]:
appRank_pay_playstore_df = pd.DataFrame(p_appRank_pay)
appRank_pay_playstore_df = appRank_pay_playstore_df.T
appRank_pay_playstore_df.columns = col
appRank_pay_playstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_pay_playstore.csv', encoding='utf-8-sig')

appRank_pay_appstore_df = pd.DataFrame(a_appRank_pay)
appRank_pay_appstore_df = appRank_pay_appstore_df.T
appRank_pay_appstore_df.columns = col
appRank_pay_appstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_pay_appstore.csv', encoding='utf-8-sig')

In [ ]:
p_sales_r = list()
for i in range(len(p_sales_rank)):
  p_sales_r.append(p_sales_rank[i])
  p_sales_r.append(p_sales_genre[i])

a_sales_r = list()
for i in range(len(a_sales_rank)):
  a_sales_r.append(a_sales_rank[i])
  a_sales_r.append(a_sales_genre[i])

In [ ]:
p_sales_rank_df = pd.DataFrame(p_sales_r)
p_sales_rank_df = p_sales_rank_df.T
p_sales_rank_df.columns = col[24:]
p_sales_rank_df.to_csv('/content/gdrive/My Drive/app_sales_rank_playstore.csv', encoding='utf-8-sig')

a_sales_rank_df = pd.DataFrame(a_sales_r)
a_sales_rank_df = a_sales_rank_df.T
a_sales_rank_df.columns = col[24:]
a_sales_rank_df.to_csv('/content/gdrive/My Drive/app_sales_rank_appstore.csv', encoding='utf-8-sig')